In [26]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc, row_number, monotonically_increasing_id
from pyspark.sql.window import Window

from pyspark.sql.functions import col, explode, collect_list, concat_ws
from pyspark.sql.functions import col, collect_list, concat_ws, sha2
from pyspark.sql.functions import regexp_replace, regexp_extract, expr, when, lit
from pyspark.sql.functions import desc, row_number, monotonically_increasing_id
from pyspark.sql.window import Window
from pyspark.sql.functions import udf, to_date, concat
from pyspark.sql.types import ArrayType, StringType, IntegerType, DoubleType
import pyspark.sql.functions as F
import os, subprocess
import re
import pyspark.sql.functions as psf

In [27]:
sparksession = (SparkSession
                .builder
                .appName("CRA-strva-landing")
                .enableHiveSupport()
                .getOrCreate())

In [14]:

def blank_as_na(x):
    return when((col(x) != "") & (col(x) != "null") , col(x)).otherwise('Not Available')

def check_free_flag(x):
    return when((col(x) =='1') , "Yes").when((col(x) =='') , "Not Available").otherwise("No")

def is_blank_then_null(x):
    return when((col(x) != "null") , col(x)).otherwise(None)


In [39]:
df_traffic_ref_raw = sparksession.read.option("header",True).option("inferSchema", "true").option("multiline","true").csv("/Data/TCCS/TrafficVolume/SourceData/*.csv")

+--------------------+----------------------------+-----------------------------+--------------+-----------------+----------------+-------------------+-----------------------+------------------+-------------+
|   Intersection Name|IntersectionCentroidLatitude|IntersectionCentroidLongitude|ApproachNumber|ApproachFromSteet| ApproachToSteet|ApproachDescription|CityMovementDescription|IntersectionSuburb|TrafficIndex
+--------------------+----------------------------+-----------------------------+--------------+-----------------+----------------+-------------------+-----------------------+------------------+-------------+
|Constitution Ave ...|                   -35.28578|                    149.13482|   Approach  1|    Coranderrk St|   Coranderrk St|        North Bound|           Towards City|              Reid|          6.0|
|Constitution Ave ...|                   -35.28578|                    149.13482|   Approach  2|    Coranderrk St|   Coranderrk St|        North Bound|           Towa

In [46]:
df_traffic_ref_raw= df_traffic_ref_raw.replace("IntersectionSuburb\r","IntersectionSuburb")


In [42]:
df_traffic_ref_raw.columns

['Intersection Name',
 'IntersectionCentroidLatitude',
 'IntersectionCentroidLongitude',
 'ApproachNumber',
 'ApproachFromSteet',
 'ApproachToSteet',
 'ApproachDescription',
 'CityMovementDescription',
 'IntersectionSuburb',
 'TrafficIndex\r']

In [41]:
    df_traffic_ref = df_traffic_ref_raw.select('Intersection Name','IntersectionCentroidLatitude', 'IntersectionCentroidLongitude', 'ApproachNumber','ApproachFromSteet', 'ApproachToSteet','ApproachDescription', 'CityMovementDescription', 'IntersectionSuburb')


In [47]:
df_traffic_ref= df_traffic_ref.withColumn("current_date",F.current_date())

In [48]:
df_traffic_ref.show()

+--------------------+----------------------------+-----------------------------+--------------+-----------------+----------------+-------------------+-----------------------+------------------+------------+
|   Intersection Name|IntersectionCentroidLatitude|IntersectionCentroidLongitude|ApproachNumber|ApproachFromSteet| ApproachToSteet|ApproachDescription|CityMovementDescription|IntersectionSuburb|current_date|
+--------------------+----------------------------+-----------------------------+--------------+-----------------+----------------+-------------------+-----------------------+------------------+------------+
|Constitution Ave ...|                   -35.28578|                    149.13482|   Approach  1|    Coranderrk St|   Coranderrk St|        North Bound|           Towards City|              Reid|  2020-10-22|
|Constitution Ave ...|                   -35.28578|                    149.13482|   Approach  2|    Coranderrk St|   Coranderrk St|        North Bound|           Toward

In [4]:
spark.sparkContext.setLogLevel("ERROR")

In [1]:
df_traffic_volume = spark.read.text("/Data/TCCS/TrafficVolume/SourceData/*.txt")

In [3]:
df_traffic_volume.columns

['value']

In [1]:
#for line in lines:
    #print(line)

In [3]:
df_ref = sparksession.sql("SELECT  intrsctn_nm, intrsctn_lat, intrsctn_long, approach_num, approach_frm_st, approach_to_st, approach_desc, city_mv_desc, intrsctn_sub FROM traffic_volume_landing.traffic_vol_ref")

In [4]:
df_ref_new = df_ref.select(regexp_replace(col("approach_num"),"\\s+", "").alias("approach_ref"),"intrsctn_nm", "intrsctn_lat", "intrsctn_long", "approach_frm_st", "approach_to_st", "approach_desc", "city_mv_desc", "intrsctn_sub")

In [11]:
df_ref_new.select("approach_ref").show()

+------------+
|approach_ref|
+------------+
|   Approach1|
|   Approach2|
|   Approach3|
|   Approach4|
|   Approach5|
|   Approach6|
|   Approach7|
|   Approach8|
|   Approach9|
|  Approach10|
|  Approach11|
|  Approach12|
|  Approach13|
|  Approach14|
+------------+



In [3]:
df_cnt = sparksession.sql("SELECT trafficvol_dt, trafficvol_day, trafficvol_ts, approach_num, trafficvol_cnt FROM traffic_volume_landing.traffic_vol_count")

In [18]:
df_cnt = df_cnt.withColumn('joined_datetime', concat(col('trafficvol_dt'),lit(' '), col('trafficvol_ts')))

In [9]:
df_cnt_new = df_cnt.select(regexp_replace(col("approach_num"), " ", "").alias("approach_pid"),to_date(col("trafficvol_dt"),"dd MMMM yyyy").alias("trafficvol_dt"),"trafficvol_day", "trafficvol_ts", "trafficvol_cnt", "approach_num")


In [5]:
df_cnt.filter(col("trafficvol_dt") == "01 February 2019").show(20)

+----------------+--------------+-------------+------------+--------------+
|   trafficvol_dt|trafficvol_day|trafficvol_ts|approach_num|trafficvol_cnt|
+----------------+--------------+-------------+------------+--------------+
|01 February 2019|        Friday|        01:00|  Approach 1|            19|
|01 February 2019|        Friday|        01:00|  Approach 2|            15|
|01 February 2019|        Friday|        01:00|  Approach 3|            34|
|01 February 2019|        Friday|        01:00|  Approach 4|            21|
|01 February 2019|        Friday|        01:00|  Approach 5|             0|
|01 February 2019|        Friday|        01:00|  Approach 6|             0|
|01 February 2019|        Friday|        01:00|  Approach 7|             8|
|01 February 2019|        Friday|        01:00|  Approach 8|             7|
|01 February 2019|        Friday|        01:00|  Approach 9|            28|
|01 February 2019|        Friday|        01:00| Approach 10|             0|
|01 February

In [15]:
df_cnt_new = df_cnt_new.withColumn('joined_datetime', concat(col('trafficvol_dt'),lit(' '), col('trafficvol_ts')))

In [ ]:
df_cnt_new

In [10]:
df_cnt_new.columns

['approach_pid',
 'trafficvol_dt',
 'trafficvol_day',
 'trafficvol_ts',
 'trafficvol_cnt',
 'approach_num']

In [11]:
df_cnt_new.show()

+------------+-------------+--------------+-------------+--------------+------------+
|approach_pid|trafficvol_dt|trafficvol_day|trafficvol_ts|trafficvol_cnt|approach_num|
+------------+-------------+--------------+-------------+--------------+------------+
|   Approach1|   2019-12-01|        Sunday|        01:00|            83|  Approach 1|
|   Approach2|   2019-12-01|        Sunday|        01:00|            53|  Approach 2|
|   Approach3|   2019-12-01|        Sunday|        01:00|            97|  Approach 3|
|   Approach4|   2019-12-01|        Sunday|        01:00|            74|  Approach 4|
|   Approach5|   2019-12-01|        Sunday|        01:00|             1|  Approach 5|
|   Approach6|   2019-12-01|        Sunday|        01:00|             8|  Approach 6|
|   Approach7|   2019-12-01|        Sunday|        01:00|            19|  Approach 7|
|   Approach8|   2019-12-01|        Sunday|        01:00|            12|  Approach 8|
|   Approach9|   2019-12-01|        Sunday|        01:

In [11]:
df_cnt_new.printSchema


<bound method DataFrame.printSchema of DataFrame[approach_num: string, trafficvol_dt: date, trafficvol_day: string, trafficvol_ts: string, trafficvol_cnt: string]>

In [12]:
df_new =df_cnt_new.join(df_ref_new,df_cnt_new.approach_pid == df_ref_new.approach_ref) 


df_new = df_new.drop(df_new.approach_pid)

df_new = df_new.drop(df_new.approach_ref)

df_new = df_new.withColumn("trafficvol_cnt", df_new["trafficvol_cnt"].cast(IntegerType()))

In [19]:
df_new.printSchema()

root
 |-- trafficvol_dt: date (nullable = true)
 |-- trafficvol_day: string (nullable = true)
 |-- trafficvol_ts: string (nullable = true)
 |-- trafficvol_cnt: integer (nullable = true)
 |-- approach_num: string (nullable = true)
 |-- intrsctn_nm: string (nullable = true)
 |-- intrsctn_lat: double (nullable = true)
 |-- intrsctn_long: double (nullable = true)
 |-- approach_frm_st: string (nullable = true)
 |-- approach_to_st: string (nullable = true)
 |-- approach_desc: string (nullable = true)
 |-- city_mv_desc: string (nullable = true)
 |-- intrsctn_sub: string (nullable = true)



In [20]:
df_new.show(30)

+-------------+--------------+-------------+--------------+------------+--------------------+------------+-------------+----------------+----------------+-------------+------------+------------+
|trafficvol_dt|trafficvol_day|trafficvol_ts|trafficvol_cnt|approach_num|         intrsctn_nm|intrsctn_lat|intrsctn_long| approach_frm_st|  approach_to_st|approach_desc|city_mv_desc|intrsctn_sub|
+-------------+--------------+-------------+--------------+------------+--------------------+------------+-------------+----------------+----------------+-------------+------------+------------+
|   2019-12-01|        Sunday|        01:00|            83|  Approach 1|Constitution Ave ...|   -35.28578|    149.13482|   Coranderrk St|   Coranderrk St|  North Bound|Towards City|        Reid|
|   2019-12-01|        Sunday|        01:00|            53|  Approach 2|Constitution Ave ...|   -35.28578|    149.13482|   Coranderrk St|   Coranderrk St|  North Bound|Towards City|        Reid|
|   2019-12-01|        Su

In [24]:
diff = df_ref.join(df_cnt,df_ref["approach_num"]==df_cnt["approach_ref"],"left")

In [25]:
diff.show()

+--------------------+------------+-------------+------------+----------------+----------------+-------------+------------+------------+-------------+--------------+-------------+------------+--------------+
|         intrsctn_nm|intrsctn_lat|intrsctn_long|approach_num| approach_frm_st|  approach_to_st|approach_desc|city_mv_desc|intrsctn_sub|trafficvol_dt|trafficvol_day|trafficvol_ts|approach_ref|trafficvol_cnt|
+--------------------+------------+-------------+------------+----------------+----------------+-------------+------------+------------+-------------+--------------+-------------+------------+--------------+
|Constitution Ave ...|   -35.28578|    149.13482| Approach  1|   Coranderrk St|   Coranderrk St|  North Bound|Towards City|        Reid|         null|          null|         null|        null|          null|
|Constitution Ave ...|   -35.28578|    149.13482| Approach  2|   Coranderrk St|   Coranderrk St|  North Bound|Towards City|        Reid|         null|          null|   

In [28]:
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [35]:
df_cnewt = df_cnt.select(regexp_replace(col("approach_ref"), " ", "").alias("cref"),"trafficvol_dt")

In [60]:
df_cnewt.select(col("trafficvol_dt").alias("trafficvoldate"), to_date(col("trafficvol_dt"),"dd MMMM yyyy").alias("Formated Date")).show()

+----------------+-------------+
|  trafficvoldate|Formated Date|
+----------------+-------------+
|01 December 2019|   2019-12-01|
|01 December 2019|   2019-12-01|
|01 December 2019|   2019-12-01|
|01 December 2019|   2019-12-01|
|01 December 2019|   2019-12-01|
|01 December 2019|   2019-12-01|
|01 December 2019|   2019-12-01|
|01 December 2019|   2019-12-01|
|01 December 2019|   2019-12-01|
|01 December 2019|   2019-12-01|
|01 December 2019|   2019-12-01|
|01 December 2019|   2019-12-01|
|01 December 2019|   2019-12-01|
|01 December 2019|   2019-12-01|
|01 December 2019|   2019-12-01|
|01 December 2019|   2019-12-01|
|01 December 2019|   2019-12-01|
|01 December 2019|   2019-12-01|
|01 December 2019|   2019-12-01|
|01 December 2019|   2019-12-01|
+----------------+-------------+
only showing top 20 rows



In [6]:
df_curate = sparksession.sql("SELECT trafficvol_dt, trafficvol_day, trafficvol_ts, approach_num, trafficvol_cnt FROM traffic_volume_curated.traffic_vol_count_curated")

In [8]:
df_curate.filter(col("trafficvol_dt") == "2019-02-01").show(50)


+-------------+--------------+-------------+------------+--------------+
|trafficvol_dt|trafficvol_day|trafficvol_ts|approach_num|trafficvol_cnt|
+-------------+--------------+-------------+------------+--------------+
|   2019-02-01|        Friday|        01:00|  Approach 1|            19|
|   2019-02-01|        Friday|        01:00|  Approach 2|            15|
|   2019-02-01|        Friday|        01:00|  Approach 3|            34|
|   2019-02-01|        Friday|        01:00|  Approach 4|            21|
|   2019-02-01|        Friday|        01:00|  Approach 5|             0|
|   2019-02-01|        Friday|        01:00|  Approach 6|             0|
|   2019-02-01|        Friday|        01:00|  Approach 7|             8|
|   2019-02-01|        Friday|        01:00|  Approach 8|             7|
|   2019-02-01|        Friday|        01:00|  Approach 9|            28|
|   2019-02-01|        Friday|        01:00| Approach 10|             0|
|   2019-02-01|        Friday|        01:00| Approa

In [22]:
!pip install geopandas

    100% |████████████████████████████████| 972kB 2.3MB/s ta 0:00:011
    100% |████████████████████████████████| 10.9MB 2.1MB/s eta 0:00:01
    100% |████████████████████████████████| 1.0MB 12.0MB/s ta 0:00:01███████         | 737kB 71.6MB/s eta 0:00:01
    100% |████████████████████████████████| 14.7MB 2.0MB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 1.7MB/s ta 0:00:01
You are using pip version 10.0.1, however version 20.2.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [23]:
shp_file = geopandas.read_file('cf485f9235bc3082bea15f504a693567edbac0a814305c849c816b909041951b-1603663006342.zip.shp')
shp_file.to_file('myshpfile.geojson', driver='GeoJSON')

NameError: name 'geopandas' is not defined

In [1]:
import geopandas

ImportError: No module named 'geopandas'

In [2]:
import shapefile
from json import dumps

# read the shapefile
reader = shapefile.Reader("cf485f9235bc3082bea15f504a693567edbac0a814305c849c816b909041951b-1603663006342.zip.shp")
fields = reader.fields[1:]
field_names = [field[0] for field in fields]
buffer = []
for sr in reader.shapeRecords():
    atr = dict(zip(field_names, sr.record))
    geom = sr.shape.__geo_interface__
    buffer.append(dict(type="Feature", \
    geometry=geom, properties=atr)) 
   
    # write the GeoJSON file
   
geojson = open("pyshp-demo.json", "w")
geojson.write(dumps({"type": "FeatureCollection", "features": buffer}, indent=2) + "\n")
geojson.close()

ImportError: No module named 'shapefile'

In [4]:
df_raw = sparksession.read.option("multiline","true").json("/Data/CMTEDD/OCDO/StravaMetroView/SourceData/*/geojson.json")

In [6]:
df_raw.show()

+--------------------+--------------------+-----------------+
|                 crs|            features|             type|
+--------------------+--------------------+-----------------+
|[[urn:ogc:def:crs...|[[[WrappedArray(W...|FeatureCollection|
|[[urn:ogc:def:crs...|[[[WrappedArray(W...|FeatureCollection|
+--------------------+--------------------+-----------------+



In [18]:
df_strava_metro_ride = sparksession.read.option("header",True).option("inferSchema", "true").option("multiline","true").csv("/Data/CMTEDD/OCDO/StravaMetroView/SourceData/cycle.csv")

In [11]:
df_strava_metro_ride= df_strava_metro_ride.withColumn("edge", df_strava_metro_ride["edgeUID"].cast(StringType()))

In [17]:
df_strava_metro_ride.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- linetype: string (nullable = true)
 |-- edgeUID: integer (nullable = true)
 |-- osmID: integer (nullable = true)
 |-- start_linestring_long: double (nullable = true)
 |-- start_linestring_lat: double (nullable = true)
 |-- end_linestring_long: double (nullable = true)
 |-- end_linestring_lat: double (nullable = true)



In [19]:
df_strava_metro_ride.show()

+---+----------+--------+--------+---------------------+--------------------+-------------------+-------------------+
|_c0|  linetype| edgeUID|   osmID|start_linestring_long|start_linestring_lat|end_linestring_long| end_linestring_lat|
+---+----------+--------+--------+---------------------+--------------------+-------------------+-------------------+
|  0|LineString|18156735|18156735|        149.134765625|  -35.28569030761719| 149.13485717773438| -35.28575134277344|
|  1|LineString|18156737|18156737|   149.13502502441406| -35.285850524902344| 149.13514709472656|-35.285919189453125|
|  2|LineString|18156738|18156738|   149.13485717773438|  -35.28575134277344| 149.13502502441406|-35.285850524902344|
|  3|LineString|18156739|18156739|   149.13511657714844|   -35.2857551574707| 149.13502502441406|-35.285850524902344|
|  4|LineString|18156740|18156740|   149.13522338867188| -35.285823822021484| 149.13511657714844|  -35.2857551574707|
|  5|LineString|18156741|18156741|   149.13516235351562|

In [29]:
df_strava_metro_ride = sparksession.read.option("header",True).option("inferSchema", "true").option("delimiter", ",").option("multiline","true").option("quote", "\"").option("escape", "\"").option("encoding", "UTF-8").csv("/Data/CMTEDD/OCDO/StravaMetroView/SourceData/cycle.csv")
df_strava_metro_ride = df_strava_metro_ride.select("edgeUID")# "start_linestring_long", "end_linestring_long","start_linestring_lat","end_linestring_lat" )
df_strava_metro_ride= df_strava_metro_ride.withColumn("edgeUID", df_strava_metro_ride["edgeUID"].cast(StringType()))
#df_strava_metro_ride.write.saveAsTable("strava_metro_landing.strava_cycle_reference", format="parquet", mode="overwrite", path="/Data/CMTEDD/OCDO/StravaMetroView/Landing/strava_cycle_reference")

In [24]:
df_strava_metro_ride.dtypes

[('linetype', 'string')]

In [15]:
df_strava_metro_ride.printSchema()

root
 |-- linetype: string (nullable = true)
 |-- edgeUID: integer (nullable = true)
 |-- osmID: integer (nullable = true)
 |-- start_linestring_long: double (nullable = true)
 |-- end_linestring_long: double (nullable = true)
 |-- start_linestring_lat: double (nullable = true)
 |-- end_linestring_lat: double (nullable = true)



In [71]:
df_cycle_cnt = sparksession.sql("SELECT *  FROM strava_metroview_landing.strava_cycle_count")
df_cycle_cnt = df_cycle_cnt.withColumnRenamed('reverse_average_speed\r', 'reverse_average_speed')

df_cycle_ref= sparksession.sql("SELECT *  FROM strava_metroview_landing.strava_cycle_reference")

In [73]:
df_cycle_ref.columns

['linetype',
 'edgeUID',
 'osmID',
 'start_linestring_long',
 'end_linestring_long',
 'start_linestring_lat',
 'end_linestring_lat']

In [74]:
df_curated =df_cycle_cnt.join(df_cycle_ref,df_cycle_cnt.edge_uid == df_cycle_ref.osmID) 
df_curated = df_curated.drop(df_curated.edgeUID)
df_curated = df_curated.drop(df_curated.osmID)



In [ ]:
edge_uid,activity_type,date, forward_trip_count, reverse_trip_count,forward_people_count,reverse_people_count,forward_hour_0_trip_count,reverse_hour_0_trip_count,forward_hour_1_trip_count,reverse_hour_1_trip_count,forward_hour_2_trip_count,reverse_hour_2_trip_count,forward_hour_3_trip_count,reverse_hour_3_trip_count,forward_hour_4_trip_count,reverse_hour_4_trip_count,forward_hour_5_trip_count,reverse_hour_5_trip_count,forward_hour_6_trip_count,reverse_hour_6_trip_count,forward_hour_7_trip_count,reverse_hour_7_trip_count,forward_hour_8_trip_count,reverse_hour_8_trip_count,forward_hour_9_trip_count,reverse_hour_9_trip_count,forward_hour_10_trip_count,reverse_hour_10_trip_count,forward_hour_11_trip_count,reverse_hour_11_trip_count,forward_hour_12_trip_count,reverse_hour_12_trip_count,forward_hour_13_trip_count,reverse_hour_13_trip_count,forward_hour_14_trip_count,reverse_hour_14_trip_count,forward_hour_15_trip_count,reverse_hour_15_trip_count,forward_hour_16_trip_count,reverse_hour_16_trip_count,forward_hour_17_trip_count,reverse_hour_17_trip_count,forward_hour_18_trip_count,reverse_hour_18_trip_count,forward_hour_19_trip_count,reverse_hour_19_trip_count,forward_hour_20_trip_count,reverse_hour_20_trip_count,forward_hour_21_trip_count,reverse_hour_21_trip_count,forward_hour_22_trip_count,reverse_hour_22_trip_count,forward_hour_23_trip_count,reverse_hour_23_trip_count,forward_commute_trip_count,reverse_commute_trip_count,forward_leisure_trip_count,reverse_leisure_trip_count,forward_morning_trip_count,reverse_morning_trip_count,forward_evening_trip_count,reverse_evening_trip_count,forward_male_people_count,reverse_male_people_count,forward_female_people_count,reverse_female_people_count,forward_unspecified_people_count,reverse_unspecified_people_count,forward_13_19_people_count,reverse_13_19_people_count,forward_20_34_people_count,reverse_20_34_people_count,forward_35_54_people_count,reverse_35_54_people_count,forward_55_64_people_count,reverse_55_64_people_count,forward_65_plus_people_count,reverse_65_plus_people_count,forward_average_speed, reverse_average_speed\r

In [55]:
 df_cycle_cnt.columns

['edge_uid',
 'activity_type',
 'date',
 'forward_trip_count',
 'reverse_trip_count',
 'forward_people_count',
 'reverse_people_count',
 'forward_hour_0_trip_count',
 'reverse_hour_0_trip_count',
 'forward_hour_1_trip_count',
 'reverse_hour_1_trip_count',
 'forward_hour_2_trip_count',
 'reverse_hour_2_trip_count',
 'forward_hour_3_trip_count',
 'reverse_hour_3_trip_count',
 'forward_hour_4_trip_count',
 'reverse_hour_4_trip_count',
 'forward_hour_5_trip_count',
 'reverse_hour_5_trip_count',
 'forward_hour_6_trip_count',
 'reverse_hour_6_trip_count',
 'forward_hour_7_trip_count',
 'reverse_hour_7_trip_count',
 'forward_hour_8_trip_count',
 'reverse_hour_8_trip_count',
 'forward_hour_9_trip_count',
 'reverse_hour_9_trip_count',
 'forward_hour_10_trip_count',
 'reverse_hour_10_trip_count',
 'forward_hour_11_trip_count',
 'reverse_hour_11_trip_count',
 'forward_hour_12_trip_count',
 'reverse_hour_12_trip_count',
 'forward_hour_13_trip_count',
 'reverse_hour_13_trip_count',
 'forward_hour_1

In [48]:
import re    

df_new_col = [re.sub(r'[\r]', '', c) for c in df_c]

In [50]:
df_cycle_cnt.columns = df_new_col 

AttributeError: can't set attribute